# INM707 Coursework - Advanced Task
# Space Invaders with Asynchronous Advantage Actor-Critic (A3C) - LSTM
### By: Elisabeta Monica Furdui: 190045971 and Jasveen Kaur: 190020638 

For this implementation, we have used OpenAI gym and Pytorch library.

In [1]:
###############
#Code referenced and corrected/edited from: https://github.com/ikostrikov/pytorch-a3c
###############

from __future__ import print_function
import cv2
import sys
import gym
import numpy as np
from gym.spaces.box import Box
from gym import wrappers
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import time
from collections import deque
import os
import torch.multiprocessing as mp
import pdb
import math
import torch.optim as optim

### Preprocessing the environment

In [ ]:
# Taken from https://github.com/openai/universe-starter-agent
def create_atari_env(env_id, video=False):
    env = gym.make(env_id)
    if video:
        env = wrappers.Monitor(env, 'test', force=True)
    env = MyAtariRescale42x42(env)
    env = MyNormalizedEnv(env)
    return env


def _process_frame42(frame):
    frame = frame[34:34 + 160, :160]
    # Resize by half, then down to 42x42 (essentially mipmapping). If
    # we resize directly we lose pixels that, when mapped to 42x42,
    # aren't close enough to the pixel boundary.
    frame = cv2.resize(frame, (80, 80))
    frame = cv2.resize(frame, (42, 42))
    frame = frame.mean(2)
    frame = frame.astype(np.float32)
    frame *= (1.0 / 255.0)
    #frame = np.reshape(frame, [1, 42, 42])
    return frame


class MyAtariRescale42x42(gym.ObservationWrapper):

    def __init__(self, env=None):
        super(MyAtariRescale42x42, self).__init__(env)
        self.observation_space = Box(0.0, 1.0, [1, 42, 42], dtype = np.float32)

    def observation(self, observation):
        return _process_frame42(observation)


class MyNormalizedEnv(gym.ObservationWrapper):

    def __init__(self, env=None):
        super(MyNormalizedEnv, self).__init__(env)
        self.state_mean = 0
        self.state_std = 0
        self.alpha = 0.9999
        self.num_steps = 0

    def observation(self, observation):
        self.num_steps += 1
        self.state_mean = self.state_mean * self.alpha + observation.mean() * (1 - self.alpha)
        self.state_std = self.state_std * self.alpha + observation.std() * (1 - self.alpha)

        unbiased_mean = self.state_mean / (1 - pow(self.alpha, self.num_steps))
        unbiased_std = self.state_std / (1 - pow(self.alpha, self.num_steps))

        ret = (observation - unbiased_mean) / (unbiased_std + 1e-8)
        return np.expand_dims(ret, axis=0)


### Shared Optimizer

In [2]:
class SharedRMSprop(optim.RMSprop):
    """Implements RMSprop algorithm with shared states.
    """

    def __init__(self, params, lr=1e-2, alpha=0.99, eps=1e-8, weight_decay=0, momentum=0, centered=False):
        super(SharedRMSprop, self).__init__(params, lr, alpha, eps, weight_decay, momentum, centered)

    def __setstate__(self, state):
        super(SharedRMSprop, self).__setstate__(state)

    def share_memory(self):
        for group in self.param_groups:
            for p in group['params']:
                state = self.state[p]
                state['square_avg'].share_memory_()
                state['step'].share_memory_()
                state['grad_avg'].share_memory_()
                state['momentum_buffer'].share_memory_()

    def step(self):
        loss = None

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data
                state = self.state[p]

                square_avg = state['square_avg']
                alpha = group['alpha']

                state['step'] += 1

                if group['weight_decay'] != 0:
                    grad = grad.add(group['weight_decay'], p.data)

                square_avg.mul_(alpha).addcmul_(1 - alpha, grad, grad)

                if group['centered']:
                    grad_avg = state['grad_avg']
                    grad_avg.mul_(alpha).add_(1 - alpha, grad)
                    avg = square_avg.addcmul(-1, grad_avg, grad_avg).sqrt().add_(group['eps'])
                else:
                    avg = square_avg.sqrt().add_(group['eps'])

                if group['momentum'] > 0:
                    buf = state['momentum_buffer']
                    buf.mul_(group['momentum']).addcdiv_(grad, avg)
                    p.data.add_(-group['lr'], buf)
                else:
                    p.data.addcdiv_(-group['lr'], grad, avg)

        return loss
    
    
class SharedAdam(optim.Adam):

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0):
        super(SharedAdam, self).__init__(params, lr, betas, eps, weight_decay)
        for group in self.param_groups:
            for p in group['params']:
                state = self.state[p]
                state['step'] = torch.zeros(1)
                state['exp_avg'] = p.data.new().resize_as_(p.data).zero_()
                state['exp_avg_sq'] = p.data.new().resize_as_(p.data).zero_()

    def share_memory(self):
        for group in self.param_groups:
            for p in group['params']:
                state = self.state[p]
                state['step'].share_memory_()
                state['exp_avg'].share_memory_()
                state['exp_avg_sq'].share_memory_()

    def step(self):
        loss = None
        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                
                grad = p.grad.data
                state = self.state[p]
                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']
                state['step'] += 1
                
                if group['weight_decay'] != 0:
                    grad = grad.add(group['weight_decay'], p.data)
               
                exp_avg.mul_(beta1).add_(1 - beta1, grad)
                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                denom = exp_avg_sq.sqrt().add_(group['eps'])
                
                bias_correction1 = 1 - beta1 ** state['step'].item()
                bias_correction2 = 1 - beta2 ** state['step'].item()
                step_size = group['lr'] * math.sqrt(bias_correction2) / bias_correction1
                p.data.addcdiv_(-step_size, exp_avg, denom)
        return loss


### Initializing the weights and defining the A3C LSTM network architecture
Since the agents share the same network, therefore they also share common weights.

In [3]:
# Initializing and setting the variance of a tensor of weights
def normalized_columns_initializer(weights, std=1.0):
    out = torch.randn(weights.size())
    out *= std / torch.sqrt(out.pow(2).sum(1, keepdim = True))
    return out

# Initializing the weights of the neural network in an optimal way for the learning
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        weight_shape = list(m.weight.data.size())
        fan_in = np.prod(weight_shape[1:4])
        fan_out = np.prod(weight_shape[2:4]) * weight_shape[0]
        w_bound = np.sqrt(6. / (fan_in + fan_out))
        m.weight.data.uniform_(-w_bound, w_bound)
        m.bias.data.fill_(0)
    elif classname.find('Linear') != -1:
        weight_shape = list(m.weight.data.size())
        fan_in = weight_shape[1]
        fan_out = weight_shape[0]
        w_bound = np.sqrt(6. / (fan_in + fan_out))
        m.weight.data.uniform_(-w_bound, w_bound)
        m.bias.data.fill_(0)

#A3C LSTM Model
class ActorCritic(torch.nn.Module):

    def __init__(self, num_inputs, action_space):
        super(ActorCritic, self).__init__()
        self.conv1 = nn.Conv2d(num_inputs, 32, 3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(32, 32, 3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(32, 32, 3, stride=2, padding=1)
        self.conv4 = nn.Conv2d(32, 32, 3, stride=2, padding=1)
        self.lstm = nn.LSTMCell(32 * 3 * 3, 256)
        num_outputs = action_space.n
        self.critic_linear = nn.Linear(256, 1)
        self.actor_linear = nn.Linear(256, num_outputs)
        self.apply(weights_init)
        self.actor_linear.weight.data = normalized_columns_initializer(self.actor_linear.weight.data, 0.01)
        self.actor_linear.bias.data.fill_(0)
        self.critic_linear.weight.data = normalized_columns_initializer(self.critic_linear.weight.data, 1.0)
        self.critic_linear.bias.data.fill_(0)
        self.lstm.bias_ih.data.fill_(0)
        self.lstm.bias_hh.data.fill_(0)
        self.train()

    def forward(self, inputs):
        inputs, (hx, cx) = inputs
        x = F.elu(self.conv1(inputs))
        x = F.elu(self.conv2(x))
        x = F.elu(self.conv3(x))
        x = F.elu(self.conv4(x))
        x = x.view(-1, 32 * 3 * 3)
        hx, cx = self.lstm(x, (hx, cx))
        x = hx
        return self.critic_linear(x), self.actor_linear(x), (hx, cx)


### Function to train the Agent

In [4]:
def ensure_shared_grads(model, shared_model):
    for param, shared_param in zip(model.parameters(), shared_model.parameters()):
        if shared_param.grad is not None:
            return
        shared_param._grad = param.grad

def train(rank, params, shared_model, optimizer):
    torch.manual_seed(params.seed + rank)
    env = create_atari_env(params.env_name)
    env.seed(params.seed + rank)
    
    model = ActorCritic(env.observation_space.shape[0], env.action_space)
    model.train()
    state = env.reset()
    state = torch.from_numpy(state)
    done = True
    
    episode_length = 0
    while True:
        episode_length += 1
        model.load_state_dict(shared_model.state_dict())
        if done:
            cx = Variable(torch.zeros(1, 256))
            hx = Variable(torch.zeros(1, 256))
        else:
            cx = Variable(cx.data)
            hx = Variable(hx.data)
        
        values = []
        log_probs = []
        rewards = []
        entropies = []
        for step in range(params.num_steps):
            value, action_values, (hx, cx) = model((Variable(state.unsqueeze(0)), (hx, cx)))
            prob = F.softmax(action_values, dim = 1)
            log_prob = F.log_softmax(action_values, dim = 1)
            entropy = -(log_prob * prob).sum(1)
            
            entropies.append(entropy)
            action = prob.multinomial(num_samples = 1).data
            log_prob = log_prob.gather(1, Variable(action))
            
            values.append(value)
            log_probs.append(log_prob)
            
            state, reward, done, _ = env.step(action.numpy())
            done = (done or episode_length >= params.max_episode_length)
            reward = max(min(reward, 1), -1)
            
            if done:
                episode_length = 0
                state = env.reset()
            
            state = torch.from_numpy(state)
            rewards.append(reward)
            if done:
                break
        
        
        R = torch.zeros(1, 1)
        if not done:
            value, _, _ = model((Variable(state.unsqueeze(0)), (hx, cx)))
            R = value.data
        
        values.append(Variable(R))
        policy_loss = 0
        value_loss = 0
        R = Variable(R)
        gae = torch.zeros(1, 1)
        for i in reversed(range(len(rewards))):
            R = params.gamma * R + rewards[i]
            advantage = R - values[i]
            value_loss = value_loss + 0.5 * advantage.pow(2)
            TD = rewards[i] + params.gamma * values[i + 1].data - values[i].data
            gae = gae * params.gamma * params.tau + TD
            policy_loss = policy_loss - log_probs[i] * Variable(gae) - 0.01 * entropies[i]
        optimizer.zero_grad()
        (policy_loss + 0.5 * value_loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 40)
        ensure_shared_grads(model, shared_model)
        optimizer.step()


### Function to test the agent's performance¶

In [5]:
def directory(params):
    # logging
    log_dir = os.path.join('logs', params.model_name)
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    
    log_filename = params.env_name+"."+ params.model_name+".log"
    f = open(os.path.join(log_dir, log_filename), "w")
    # model saver
    ckpt_dir = os.path.join('ckpt', params.model_name)
    
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)
    ckpt_filename = params.env_name +"."+ params.model_name +".pkl"
    return (f, os.path.join(ckpt_dir, ckpt_filename)), (log_dir, ckpt_dir)

def test(rank, params, shared_model):
    torch.manual_seed(params.seed + rank)
    env = create_atari_env(params.env_name, video=True)
    env.seed(params.seed + rank)
    
    model = ActorCritic(env.observation_space.shape[0], env.action_space)
    model.eval()
    
    
    (f, ckpt_path), (log_dir, ckpt_dir) = directory(params)
    
    state = env.reset()
    state = torch.from_numpy(state)
    
    reward_sum = 0
    
    if params.task == 'eval':
        reward_list = []
    
    done = True 
    #env = wrappers.Monitor(env, '/tmp/{}-experiment'.format(params.env_name), force=True)
    start_time = time.time()
    actions = deque(maxlen=100)
    episode_i = 0
    episode_length = 0
    
    while True:
        episode_length += 1
        if done:
            model.load_state_dict(shared_model.state_dict())
            with torch.no_grad():
                cx = Variable(torch.zeros(1, 256))
                hx = Variable(torch.zeros(1, 256))
        else:
            with torch.no_grad():
                cx = Variable(cx.data)
                hx = Variable(hx.data)
        
        with torch.no_grad():
            value, action_value, (hx, cx) = model((Variable(state.unsqueeze(0)), (hx, cx)))
        
        prob = F.softmax(action_value, dim = 1)
        action = prob.max(1)[1].data.numpy()
        state, reward, done, _ = env.step(action[0])
        done = done or episode_length >= params.max_episode_length
        reward_sum += reward
        
        if done:
            episode_i += 1
            if params.task == 'eval':
                reward_list.append(reward_sum)
            if params.task == 'eval' and episode_i >= 100:
                print ("Testing over %d episodes, Average reward = %f" % (episode_i, sum(reward_list)/episode_i,))
                break
            
            if episode_i % params.save_freq == 0:
                torch.save(model.state_dict(), os.path.join(ckpt_dir, params.env_name + "." + params.model_name +"." + str(episode_i) + ".pkl"))
        
            info = "Time {}, episode reward {}, episode length {}".format(time.strftime("%Hh %Mm %Ss", time.gmtime(time.time() - start_time)), reward_sum, episode_length)
            print(info)
            f.write(info + '\n')
            reward_sum = 0
            episode_length = 0
            actions.clear()
            state = env.reset()
            if params.task == 'train':    
                time.sleep(60)
        state = torch.from_numpy(state)
   
    env.close()
    f.close()
    torch.save(model.state_dict(), ckpt_path)

### Defining the parameters used

In [7]:
class Params():
    def __init__(self):
        self.lr = 0.0001
        self.gamma = 0.99
        self.tau = 1.
        self.seed = 1
        self.num_processes = mp.cpu_count()
        self.num_steps = 20
        self.max_episode_length = 4000
        self.env_name = 'SpaceInvaders-v0'
        self.task = 'eval'
        self.save_freq = 20
        self.model_name = 'ActorCritic'
        self.load_ckpt = 'ckpt/ActorCritic/SpaceInvaders-v0.ActorCritic.5200.pkl'


### Running all the agents in parallel with 7 processors

In [ ]:
if __name__ == '__main__':
    os.environ['OMP_NUM_THREADS'] = '1'
    params = Params()
    torch.manual_seed(params.seed)
    env = create_atari_env(params.env_name)
        
    shared_model = ActorCritic(env.observation_space.shape[0], env.action_space)
    shared_model.share_memory()
    optimizer = SharedAdam(shared_model.parameters(), lr=params.lr)
    optimizer.share_memory()
    
    if params.task == 'train':
        processes = []
        p = mp.Process(target=test, args=(params.num_processes, params, shared_model))
        p.start()
        processes.append(p)
            
        for rank in range(0, params.num_processes):
            p = mp.Process(target=train, args=(rank, params, shared_model, optimizer))
            p.start()
            processes.append(p)
            
        for p in processes:
            p.join()
        
    elif params.task == 'eval':
        shared_model.load_state_dict(torch.load(params.load_ckpt))
        test(params.num_processes, params, shared_model)
        
    elif params.task == 'develop':
        train(0, params, shared_model, optimizer)


/home/jas/ENTER/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
/opt/conda/conda-bld/pytorch_1587428266983/work/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)
/opt/conda/conda-bld/pytorch_1587428266983/work/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)
/opt/conda/conda-bld/pytorch_1587428266983/work/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number al

Time 00h 00m 05s, episode reward 115.0, episode length 491
Time 00h 01m 13s, episode reward 250.0, episode length 731
Time 00h 02m 20s, episode reward 270.0, episode length 820
Time 00h 03m 26s, episode reward 210.0, episode length 836
Time 00h 04m 34s, episode reward 285.0, episode length 959
Time 00h 05m 40s, episode reward 270.0, episode length 718
Time 00h 06m 46s, episode reward 270.0, episode length 718
Time 00h 07m 53s, episode reward 285.0, episode length 961
Time 00h 09m 11s, episode reward 595.0, episode length 1405
Time 00h 10m 19s, episode reward 285.0, episode length 973
Time 00h 11m 26s, episode reward 385.0, episode length 1040
Time 00h 12m 31s, episode reward 270.0, episode length 725
Time 00h 13m 37s, episode reward 270.0, episode length 710
Time 00h 14m 43s, episode reward 270.0, episode length 719
Time 00h 15m 49s, episode reward 270.0, episode length 716
Time 00h 16m 55s, episode reward 270.0, episode length 727
Time 00h 18m 00s, episode reward 270.0, episode length

Time 02h 31m 29s, episode reward 365.0, episode length 844
Time 02h 32m 36s, episode reward 365.0, episode length 679
Time 02h 33m 44s, episode reward 325.0, episode length 819
Time 02h 34m 53s, episode reward 465.0, episode length 947
Time 02h 36m 01s, episode reward 440.0, episode length 929
Time 02h 37m 09s, episode reward 420.0, episode length 847
Time 02h 38m 16s, episode reward 575.0, episode length 810
Time 02h 39m 24s, episode reward 365.0, episode length 876
Time 02h 40m 32s, episode reward 395.0, episode length 795
Time 02h 41m 38s, episode reward 420.0, episode length 790
Time 02h 42m 47s, episode reward 370.0, episode length 845
Time 02h 43m 54s, episode reward 315.0, episode length 777
Time 02h 45m 03s, episode reward 485.0, episode length 897
Time 02h 46m 12s, episode reward 365.0, episode length 873
Time 02h 47m 20s, episode reward 390.0, episode length 770
Time 02h 48m 31s, episode reward 330.0, episode length 789
Time 02h 49m 40s, episode reward 365.0, episode length 7

Time 05h 10m 20s, episode reward 590.0, episode length 818
Time 05h 11m 30s, episode reward 370.0, episode length 911
Time 05h 12m 38s, episode reward 365.0, episode length 760
Time 05h 13m 48s, episode reward 525.0, episode length 954
Time 05h 14m 57s, episode reward 395.0, episode length 860
Time 05h 16m 05s, episode reward 360.0, episode length 852
Time 05h 17m 13s, episode reward 640.0, episode length 808
Time 05h 18m 23s, episode reward 420.0, episode length 839
Time 05h 19m 32s, episode reward 420.0, episode length 884
Time 05h 20m 39s, episode reward 330.0, episode length 765
Time 05h 21m 50s, episode reward 560.0, episode length 801
Time 05h 23m 02s, episode reward 420.0, episode length 933
Time 05h 24m 12s, episode reward 305.0, episode length 846
Time 05h 25m 23s, episode reward 305.0, episode length 696
Time 05h 27m 05s, episode reward 390.0, episode length 801
Time 05h 28m 46s, episode reward 390.0, episode length 816
Time 05h 33m 46s, episode reward 465.0, episode length 1

Time 08h 21m 46s, episode reward 575.0, episode length 1109
Time 08h 22m 53s, episode reward 510.0, episode length 1106
Time 08h 23m 58s, episode reward 365.0, episode length 774
Time 08h 25m 05s, episode reward 505.0, episode length 1124
Time 08h 26m 11s, episode reward 545.0, episode length 1070
Time 08h 27m 15s, episode reward 350.0, episode length 668
Time 08h 28m 19s, episode reward 355.0, episode length 803
Time 08h 29m 23s, episode reward 420.0, episode length 752
Time 08h 30m 28s, episode reward 460.0, episode length 1000
Time 08h 31m 31s, episode reward 260.0, episode length 458
Time 08h 32m 33s, episode reward 95.0, episode length 431
Time 08h 33m 37s, episode reward 280.0, episode length 690
Time 08h 34m 40s, episode reward 260.0, episode length 495
Time 08h 35m 46s, episode reward 605.0, episode length 1267
Time 08h 36m 49s, episode reward 260.0, episode length 639
Time 08h 37m 54s, episode reward 500.0, episode length 1009
Time 08h 38m 57s, episode reward 210.0, episode le

...

Time 07h 45m 00s, episode reward 260.0, episode length 539
Time 07h 46m 04s, episode reward 255.0, episode length 555
Time 07h 47m 10s, episode reward 340.0, episode length 752
Time 07h 48m 16s, episode reward 450.0, episode length 847
Time 07h 49m 22s, episode reward 470.0, episode length 950
Time 07h 50m 33s, episode reward 855.0, episode length 1614
Time 07h 51m 40s, episode reward 460.0, episode length 989
Time 07h 52m 46s, episode reward 390.0, episode length 824
Time 07h 53m 52s, episode reward 435.0, episode length 913
Time 07h 55m 01s, episode reward 575.0, episode length 1244
Time 07h 56m 07s, episode reward 470.0, episode length 958
Time 07h 57m 12s, episode reward 375.0, episode length 725
Time 07h 58m 20s, episode reward 545.0, episode length 1063
Time 07h 59m 26s, episode reward 370.0, episode length 759
Time 08h 00m 31s, episode reward 430.0, episode length 731
Time 08h 01m 38s, episode reward 460.0, episode length 905
Time 08h 02m 43s, episode reward 390.0, episode lengt

Time 10h 11m 54s, episode reward 640.0, episode length 1306
Time 10h 12m 56s, episode reward 305.0, episode length 687
Time 10h 13m 58s, episode reward 380.0, episode length 806
Time 10h 15m 00s, episode reward 525.0, episode length 907
Time 10h 16m 02s, episode reward 380.0, episode length 799
Time 10h 17m 04s, episode reward 345.0, episode length 888
Time 10h 18m 06s, episode reward 525.0, episode length 1031
Time 10h 19m 08s, episode reward 435.0, episode length 910
Time 10h 20m 11s, episode reward 700.0, episode length 1227
Time 10h 21m 14s, episode reward 490.0, episode length 1070
Time 10h 22m 15s, episode reward 400.0, episode length 717
Time 10h 23m 18s, episode reward 670.0, episode length 987
Time 10h 24m 19s, episode reward 305.0, episode length 642
Time 10h 25m 22s, episode reward 490.0, episode length 995
Time 10h 26m 23s, episode reward 435.0, episode length 785
Time 10h 27m 25s, episode reward 380.0, episode length 783
Time 10h 28m 28s, episode reward 715.0, episode leng

Time 12h 43m 41s, episode reward 385.0, episode length 833
Time 12h 44m 46s, episode reward 405.0, episode length 749
Time 12h 45m 51s, episode reward 405.0, episode length 817
Time 12h 46m 59s, episode reward 610.0, episode length 1151
Time 12h 48m 03s, episode reward 260.0, episode length 541
Time 12h 49m 09s, episode reward 470.0, episode length 847
Time 12h 50m 16s, episode reward 490.0, episode length 947
Time 12h 51m 22s, episode reward 305.0, episode length 763
Time 12h 52m 31s, episode reward 570.0, episode length 1338
Time 12h 53m 39s, episode reward 520.0, episode length 1161
Time 12h 54m 45s, episode reward 575.0, episode length 963
Time 12h 55m 51s, episode reward 570.0, episode length 953
Time 12h 56m 57s, episode reward 430.0, episode length 761
Time 12h 58m 05s, episode reward 540.0, episode length 1175
Time 12h 59m 11s, episode reward 570.0, episode length 1008
Time 13h 00m 18s, episode reward 470.0, episode length 985
Time 13h 01m 24s, episode reward 440.0, episode len

Time 15h 10m 48s, episode reward 335.0, episode length 614
Time 15h 11m 50s, episode reward 490.0, episode length 989
Time 15h 12m 53s, episode reward 515.0, episode length 1147
Time 15h 13m 54s, episode reward 275.0, episode length 553
Time 15h 14m 56s, episode reward 365.0, episode length 737
Time 15h 15m 58s, episode reward 530.0, episode length 1161
Time 15h 17m 00s, episode reward 365.0, episode length 787
Time 15h 18m 03s, episode reward 775.0, episode length 1277
Time 15h 19m 05s, episode reward 400.0, episode length 711
Time 15h 20m 07s, episode reward 670.0, episode length 951
Time 15h 21m 09s, episode reward 470.0, episode length 916
Time 15h 22m 11s, episode reward 530.0, episode length 899
Time 15h 23m 13s, episode reward 275.0, episode length 639
Time 15h 24m 15s, episode reward 600.0, episode length 1208
Time 15h 25m 17s, episode reward 510.0, episode length 915
Time 15h 26m 20s, episode reward 550.0, episode length 953
Time 15h 27m 21s, episode reward 225.0, episode leng

Time 17h 33m 55s, episode reward 470.0, episode length 1181
Time 17h 34m 58s, episode reward 575.0, episode length 1229
Time 17h 36m 00s, episode reward 485.0, episode length 1025
Time 17h 37m 03s, episode reward 550.0, episode length 987
Time 17h 38m 05s, episode reward 500.0, episode length 864
Time 17h 39m 08s, episode reward 635.0, episode length 1296
Time 17h 40m 10s, episode reward 570.0, episode length 1049
Time 17h 41m 12s, episode reward 610.0, episode length 1036
Time 17h 42m 15s, episode reward 545.0, episode length 1022
Time 17h 43m 16s, episode reward 395.0, episode length 774
Time 17h 44m 18s, episode reward 420.0, episode length 770
Time 17h 45m 20s, episode reward 410.0, episode length 766
Time 17h 46m 22s, episode reward 435.0, episode length 873
Time 17h 47m 25s, episode reward 440.0, episode length 1124
Time 17h 48m 27s, episode reward 665.0, episode length 1094
Time 17h 49m 29s, episode reward 245.0, episode length 578
Time 17h 50m 32s, episode reward 250.0, episode